In [ ]:
import os
import torch
import cv2
import numpy as np

# Load YOLOv5 model from the local clone
model = torch.hub.load('yolov5', 'yolov5s', source='local')

def detect_lanes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=50)
    
    if lines is not None:
        lanes = [line[0] for line in lines]
    else:
        lanes = []
    
    return lanes

def assign_vehicle_to_lane(vehicle, lanes):
    x_center = (vehicle['xmin'] + vehicle['xmax']) / 2
    y_center = (vehicle['ymin'] + vehicle['ymax']) / 2
    
    for lane in lanes:
        x1, y1, x2, y2 = lane
        if x1 <= x_center <= x2 or x2 <= x_center <= x1:
            return lane
    return None

def calculate_traffic_density(lanes, vehicle_lane_assignments):
    lane_density = {tuple(lane): 0 for lane in lanes}
    
    for vehicle, lane in vehicle_lane_assignments:
        if lane is not None:
            lane_density[tuple(lane)] += 1
    
    return lane_density

def identify_high_traffic_lanes(traffic_density, threshold=5):
    return {lane: count for lane, count in traffic_density.items() if count > threshold}

def show_final_traffic_density(image, lanes, traffic_density, high_traffic_lanes):
    """
    Visualize the lanes and traffic density on the image, highlighting high traffic lanes.
    """
    # Draw lanes and highlight high traffic lanes
    for lane in lanes:
        x1, y1, x2, y2 = lane
        color = (0, 255, 0)  # Default color for lanes
        if tuple(lane) in high_traffic_lanes:  # Check if lane has high traffic
            color = (0, 0, 255)  # Change color to red for high traffic
        
        cv2.line(image, (x1, y1), (x2, y2), color, 2)
    
    # Display traffic density as text near the lane
    for lane, density in traffic_density.items():
        x1, y1, x2, y2 = lane
        text_position = (x1, y1 - 10)
        cv2.putText(image, f"Density: {density}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Save the final image with lanes and traffic density
    cv2.imwrite("final_traffic_density_output.png", image)

# Function to process images from a specified directory
def process_images(image_dir):
    total_lane_density = {}
    all_lanes = []
    
    for filename in os.listdir(image_dir):
        if filename.endswith('.npy'):
            image_path = os.path.join(image_dir, filename)
            
            # Load .npy image
            image = np.load(image_path)
            
            # Ensure the image is in uint8 format for OpenCV
            if image.dtype == np.float64:
                image = (image * 255).astype(np.uint8)
            
            # Convert the image to BGR for YOLOv5
            image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Step 1: Lane detection
            lanes = detect_lanes(image_bgr)
            all_lanes.extend(lanes)

            # Step 2: Perform vehicle detection
            results = model(image_bgr)
            vehicles = results.pandas().xyxy[0]
            
            # Filter vehicles by class (car, motorcycle, bus, truck)
            vehicle_classes = [2, 3, 5, 7]
            vehicles = vehicles[vehicles['class'].isin(vehicle_classes)]
            
            # Assign vehicles to lanes
            vehicle_lane_assignments = []
            for _, vehicle in vehicles.iterrows():
                assigned_lane = assign_vehicle_to_lane(vehicle, lanes)
                vehicle_lane_assignments.append((vehicle, assigned_lane))
            
            # Calculate traffic density in each lane
            traffic_density = calculate_traffic_density(lanes, vehicle_lane_assignments)
            
            # Aggregate traffic density across images
            for lane, count in traffic_density.items():
                if lane in total_lane_density:
                    total_lane_density[lane] += count
                else:
                    total_lane_density[lane] = count

    return total_lane_density, list(set(all_lanes))

# Specify directories for training dataset
train_image_dir = r'C:\Users\prana\Downloads\archive (5)\data\train\image'

# Process training images
train_lane_density, all_lanes = process_images(train_image_dir)

# Identify high traffic lanes in the training dataset
train_high_traffic_lanes = identify_high_traffic_lanes(train_lane_density, threshold=5)

# Create a blank image for displaying results
final_image = np.zeros((720, 1280, 3), dtype=np.uint8)  # Adjust dimensions as needed

# Draw lanes and highlight high traffic lanes
for lane in all_lanes:
    x1, y1, x2, y2 = lane
    color = (0, 255, 0)  # Default color for lanes
    if tuple(lane) in train_high_traffic_lanes:  # Check if lane has high traffic
        color = (0, 0, 255)  # Change color to red for high traffic
    
    cv2.line(final_image, (x1, y1), (x2, y2), color, 2)

# Display traffic density as text near the lane
for lane, density in train_lane_density.items():
    x1, y1, x2, y2 = lane
    text_position = (x1, y1 - 10)
    cv2.putText(final_image, f"Density: {density}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Save the final image
cv2.imwrite("final_traffic_density_output.png", final_image)

# Optionally display the final image
cv2.imshow("Final Traffic Density", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Final output saved as 'final_traffic_density_output.png'")


YOLOv5  2024-9-28 Python-3.12.5 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
c:\Users\prana\Downloads\archive (5)\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\prana\Downloads\archive (5)\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\prana\Downloads\archive (5)\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\prana\Downloads\archive (5)\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args

TypeError: unhashable type: 'numpy.ndarray'